# Task 1 – Element-wise Operations on a Matrix

Objective: Implement a program to perform three separate operations on a 2D matrix using CUDA:
  1. Add a scalar to each element.
  2. Multiply each element by a scalar.
  3. Compute the square of each element.

The main program should:

  1. Initialize a 3x3 matrix with sample values.
  2. Use CUDA to compute the results for:
  • Adding a scalar value (e.g., +2).
  • Multiplying by a scalar (e.g., *3).
  • Squaring each element.
  3. Print the original matrix and results for each operation.

In [1]:
%%writefile task1.cu

#include <stdio.h>
#include <cuda_runtime.h>

#define N 3 // Define the size of the matrix (3x3)

// CUDA Kernel to add a scalar to each element
__global__ void addScalar(int *matrix, int scalar, int *result) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N * N) {
        result[idx] = matrix[idx] + scalar;
    }
}

// CUDA Kernel to multiply each element by a scalar
__global__ void multiplyScalar(int *matrix, int scalar, int *result) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N * N) {
        result[idx] = matrix[idx] * scalar;
    }
}

// CUDA Kernel to square each element
__global__ void squareElements(int *matrix, int *result) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N * N) {
        result[idx] = matrix[idx] * matrix[idx];
    }
}

void printMatrix(const char *message, int *matrix) {
    printf("%s\n", message);
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", matrix[i * N + j]);
        }
        printf("\n");
    }
    printf("\n");
}

int main() {
    int h_matrix[N * N] = {1, 2, 3, 4, 5, 6, 7, 8, 9}; // Input matrix
    int h_addResult[N * N], h_mulResult[N * N], h_squareResult[N * N]; // Result matrices
    int *d_matrix, *d_addResult, *d_mulResult, *d_squareResult; // Device pointers

    // Allocate memory on GPU
    cudaMalloc((void **)&d_matrix, N * N * sizeof(int));
    cudaMalloc((void **)&d_addResult, N * N * sizeof(int));
    cudaMalloc((void **)&d_mulResult, N * N * sizeof(int));
    cudaMalloc((void **)&d_squareResult, N * N * sizeof(int));

    // Copy input matrix to GPU
    cudaMemcpy(d_matrix, h_matrix, N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Launch kernels
    int blockSize = 9; // Number of threads per block
    int gridSize = (N * N + blockSize - 1) / blockSize; // Number of blocks

    addScalar<<<gridSize, blockSize>>>(d_matrix, 2, d_addResult);
    multiplyScalar<<<gridSize, blockSize>>>(d_matrix, 3, d_mulResult);
    squareElements<<<gridSize, blockSize>>>(d_matrix, d_squareResult);

    // Copy results back to host
    cudaMemcpy(h_addResult, d_addResult, N * N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_mulResult, d_mulResult, N * N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_squareResult, d_squareResult, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print results
    printMatrix("Input Matrix:", h_matrix);
    printMatrix("Added Scalar (+2):", h_addResult);
    printMatrix("Multiplied Scalar (*3):", h_mulResult);
    printMatrix("Squared:", h_squareResult);

    // Free GPU memory
    cudaFree(d_matrix);
    cudaFree(d_addResult);
    cudaFree(d_mulResult);
    cudaFree(d_squareResult);

    return 0;
}


Writing task1.cu


In [3]:
!nvcc task1.cu -o task1
!./task1

Input Matrix:
1 2 3 
4 5 6 
7 8 9 

Added Scalar (+2):
3 4 5 
6 7 8 
9 10 11 

Multiplied Scalar (*3):
3 6 9 
12 15 18 
21 24 27 

Squared:
1 4 9 
16 25 36 
49 64 81 



# Task 2 – Text Analysis Using GPU

Use CUDA to analyze a string and compute:
  1. Number of words.
  2. Number of sentences.
  3. Number of paragraphs.


The main program should:
  1. Initialize a string with sample text.
  2. Use CUDA to compute:
  • Words (space-separated sequences).
  • Sentences (. or ! or ?).
  • Paragraphs (separated by \n\n).
  3. Print the results.

In [5]:
%%writefile task2.cu

#include <stdio.h>
#include <cuda_runtime.h>
#include <string.h>

#define MAX_TEXT_LENGTH 1024

// CUDA Kernel to count words
__global__ void countWords(char *text, int *wordCount) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (text[idx] == '\0') return; // End of string

    if ((idx == 0 || text[idx - 1] == ' ' || text[idx - 1] == '\n') && text[idx] != ' ' && text[idx] != '\n') {
        atomicAdd(wordCount, 1);
    }
}

// CUDA Kernel to count sentences
__global__ void countSentences(char *text, int *sentenceCount) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (text[idx] == '\0') return; // End of string

    if (text[idx] == '.' || text[idx] == '!' || text[idx] == '?') {
        atomicAdd(sentenceCount, 1);
    }
}

// CUDA Kernel to count paragraphs
__global__ void countParagraphs(char *text, int *paragraphCount) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (text[idx] == '\0') return; // End of string

    if (idx == 0 && text[idx] != '\n') {
        atomicAdd(paragraphCount, 1); // First paragraph
    } else if (text[idx] == '\n' && text[idx + 1] == '\n') {
        atomicAdd(paragraphCount, 1); // Double newline
    }
}

int main() {
    // Sample text
    const char *sampleText = "This is a sentence. This is another one! Here is a new paragraph.\n\n This is a new line.";
    int textLength = strlen(sampleText);

    // Host variables for results
    int wordCount = 0, sentenceCount = 0, paragraphCount = 0;
    int *d_wordCount, *d_sentenceCount, *d_paragraphCount;
    char *d_text;

    // Allocate memory on GPU
    cudaMalloc((void **)&d_text, textLength + 1);
    cudaMalloc((void **)&d_wordCount, sizeof(int));
    cudaMalloc((void **)&d_sentenceCount, sizeof(int));
    cudaMalloc((void **)&d_paragraphCount, sizeof(int));

    // Initialize GPU result variables
    cudaMemset(d_wordCount, 0, sizeof(int));
    cudaMemset(d_sentenceCount, 0, sizeof(int));
    cudaMemset(d_paragraphCount, 0, sizeof(int));

    // Copy text to GPU
    cudaMemcpy(d_text, sampleText, textLength + 1, cudaMemcpyHostToDevice);

    // Launch kernels
    int blockSize = 256; // Threads per block
    int gridSize = (textLength + blockSize - 1) / blockSize;

    countWords<<<gridSize, blockSize>>>(d_text, d_wordCount);
    countSentences<<<gridSize, blockSize>>>(d_text, d_sentenceCount);
    countParagraphs<<<gridSize, blockSize>>>(d_text, d_paragraphCount);

    // Copy results back to host
    cudaMemcpy(&wordCount, d_wordCount, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&sentenceCount, d_sentenceCount, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&paragraphCount, d_paragraphCount, sizeof(int), cudaMemcpyDeviceToHost);

    // Print results
    printf("Text:\n%s\n\n", sampleText);
    printf("Word Count: %d\n", wordCount);
    printf("Sentence Count: %d\n", sentenceCount);
    printf("Paragraph Count: %d\n", paragraphCount);

    // Free GPU memory
    cudaFree(d_text);
    cudaFree(d_wordCount);
    cudaFree(d_sentenceCount);
    cudaFree(d_paragraphCount);

    return 0;
}


Writing task2.cu


In [6]:
!nvcc task2.cu -o task2
!./task2

Text:
This is a sentence. This is another one! Here is a new paragraph.

This is a new line.

Word Count: 18
Sentence Count: 4
Paragraph Count: 2


# Task 3 – Activation Functions on a Vector

Implement a CUDA program to apply two activation functions to a vector concurrently:

  1. Sigmoid: Outputs values between 0 and 1, commonly used for binary classification.


  2. Tanh (Hyperbolic Tangent): Output values between -1 and 1, used for non-linear activation.


The main program should:
  1. Initialize a vector of size 8 with a mix of positive and negative values.
  2. Perform Sigmoid and Tanh computations concurrently using CUDA kernels.
  3. Display the input vector, the Sigmoid activation result, and the Tanh activation result.

In [7]:
%%writefile task3.cu

#include <stdio.h>
#include <cuda_runtime.h>
#include <math.h>

// Sigmoid Activation Kernel
__global__ void sigmoidKernel(float *input, float *output, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        output[idx] = 1.0f / (1.0f + expf(-input[idx])); // Sigmoid formula
    }
}

// Tanh Activation Kernel
__global__ void tanhKernel(float *input, float *output, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        output[idx] = tanhf(input[idx]); // Tanh formula
    }
}

int main() {
    // Initialize input vector
    float h_input[] = {-2.5, 0.0, 3.2, -1.7, 4.5, 6.0, -0.5, 2.1};
    int n = sizeof(h_input) / sizeof(h_input[0]);

    // Host arrays for results
    float h_sigmoidOutput[n], h_tanhOutput[n];

    // Device arrays
    float *d_input, *d_sigmoidOutput, *d_tanhOutput;

    // Allocate memory on GPU
    cudaMalloc((void **)&d_input, n * sizeof(float));
    cudaMalloc((void **)&d_sigmoidOutput, n * sizeof(float));
    cudaMalloc((void **)&d_tanhOutput, n * sizeof(float));

    // Copy input vector to GPU
    cudaMemcpy(d_input, h_input, n * sizeof(float), cudaMemcpyHostToDevice);

    // Launch kernels
    int blockSize = 256; // Threads per block
    int gridSize = (n + blockSize - 1) / blockSize; // Blocks required

    sigmoidKernel<<<gridSize, blockSize>>>(d_input, d_sigmoidOutput, n);
    tanhKernel<<<gridSize, blockSize>>>(d_input, d_tanhOutput, n);

    // Copy results back to CPU
    cudaMemcpy(h_sigmoidOutput, d_sigmoidOutput, n * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_tanhOutput, d_tanhOutput, n * sizeof(float), cudaMemcpyDeviceToHost);

    // Print results
    printf("Input Vector:\n");
    for (int i = 0; i < n; i++) {
        printf("%.2f ", h_input[i]);
    }
    printf("\n\nSigmoid Activation Output:\n");
    for (int i = 0; i < n; i++) {
        printf("%.4f ", h_sigmoidOutput[i]);
    }
    printf("\n\nTanh Activation Output:\n");
    for (int i = 0; i < n; i++) {
        printf("%.4f ", h_tanhOutput[i]);
    }
    printf("\n");

    // Free GPU memory
    cudaFree(d_input);
    cudaFree(d_sigmoidOutput);
    cudaFree(d_tanhOutput);

    return 0;
}


Writing task3.cu


In [8]:
!nvcc task3.cu -o task3
!./task3

Input Vector:
-2.50 0.00 3.20 -1.70 4.50 6.00 -0.50 2.10 

Sigmoid Activation Output:
0.0759 0.5000 0.9608 0.1545 0.9890 0.9975 0.3775 0.8909 

Tanh Activation Output:
-0.9866 0.0000 0.9967 -0.9354 0.9998 1.0000 -0.4621 0.9705 
